In [2]:
import glob
import shutil
import SimpleITK as sitk
import numpy as np
import os
from collections import Counter
from multiprocessing import Pool
from numpy.lib.npyio import save
from numpy.lib.shape_base import _dstack_dispatcher
from numpy.lib.type_check import imag
from pathlib import Path
import pathlib
import multiprocessing
import math
import functools
from functools import partial
from Utils import *
import pandas as pd

按照特定的一些套路，统计所有的标签信息

In [44]:
original_path='/data/wyh_data/影像数据EGFR1第二部分'
special_path='/data/wyh_data/影像数据EGFR1第二部分/Marked/POSITIVE/3-1marked1'



res=get_file(original_path)
temp=filter(lambda x:Path(x).suffix in ['.gz'],res)
temp=filter(lambda x: 'Mask' in x,res)
label1path=list(temp)

special=get_file(special_path)
temp=filter(lambda x:Path(x).suffix in ['.nii'],special)
label2path=list(temp)
label=pd.DataFrame(label1path+label2path,columns=['mask_path'])
label.drop_duplicates(subset=['mask_path'],inplace=True)
label.to_csv('/home/wyh21/AI_Lung_node/data_processing/label.csv',index=False)

#用来统计所有标签,提取所有关键信息, 几行破代码，胡乱试，用了一下午，说到底不了解Python
#的机制，差的太多了


In [62]:
label=pd.read_csv('/home/wyh21/AI_Lung_node/data_processing/label.csv')#基本上所有的数据都统计全了

In [64]:
def detect_dcm_or_nii(patient_path):
    dir=Path(patient_path).parents[0]
    temp=list(map(lambda x:x.suffix,dir.iterdir()))
    if '.dcm' in temp:
        return 'dcm'
    else:
        return 'nii'
        
label['img_type']=label['mask_path'].apply(lambda x:detect_dcm_or_nii(x))
label['label_type']=label['mask_path'].apply(lambda x:Path(x).suffix[1:])
label.to_csv('/home/wyh21/AI_Lung_node/data_processing/label.csv',index=False)

In [3]:

def read_and_convert_nii_to_array(nii_path):
    img = sitk.ReadImage(nii_path)
    img_array = sitk.GetArrayFromImage(img)
    return img_array


def match_series_mask(DirectoryPath):
    '''
    对路径下的mask和影像分别存储
    '''
    path = DirectoryPath + "-nii\\*"
    nii_file_folder = glob.glob(path)
    mask_file_folder=list(filter(lambda x:'mask' in x,nii_file_folder))
    nii_file_new_folder=list(filter(lambda x:'mask' not in x,nii_file_folder))
    # 读取mask的张数

    count = 0

    for mask_file in mask_file_folder:
 
        mask_img = read_and_convert_nii_to_array(mask_file)
        # todo 将序列文件和标注文件匹配
        for slice_file in nii_file_new_folder:

            slice_img = read_and_convert_nii_to_array(slice_file)
            if slice_img.shape == mask_img.shape:
                count += 1
    return count


def get_context_nodule_coordinate(*params,ratio=1.5):#params一般是6个坐标
    return [calculate_axis_coordinate(x,y) for x,y in zip(params[::2], params[1::2])]

def calculate_axis_coordinate(a,b,ratio):
    return (a+b+ratio*a-ratio*b)//2,(a+b+ratio*b-ratio*a)//2


def classify_nodule_and_relabel(valid_z_pathes,img_array,mask_array):
    '''
    object as the fomal params,the origin num will be changed
    '''
    nodule_start=1
    for i in range(1,len(valid_z_pathes)):
        if valid_z_pathes[i]-valid_z_pathes[i-1]>1:
            nodule_start+=1
            mask_array[valid_z_pathes[i]][ mask_array[valid_z_pathes[i]]==1]=nodule_start
        else:
            if valid_z_pathes[i]-valid_z_pathes[i-1]==1:#这是层数挨着的patch
                if np.isin(1,mask_array[valid_z_pathes[i]]==img_array[valid_z_pathes[i-1]]):#证明两个patch是一个结节的
        
                    mask_array[i][ mask_array[valid_z_pathes[i]]==1]=nodule_start
                else:
                    nodule_start+=1
                    mask_array[valid_z_pathes[i]][mask_array[valid_z_pathes[i]]==1]=nodule_start
    return i


def save_np_array(array,path):#将文件上一级目录下的文件夹创建，然后保存文件夹
    path.parents[0].mkdir(parents=True, exist_ok=True)
    np.save(path,array)
    
def make_dataset(DirectoryPath:Path,slice_path_list,mask_path_list,ratio=1.5):
    '''
    classification task:crop 3D cube and 3D cube with context information
    segmatation task: use point cloud as the segmatation label
    3D detection task: save xmin xmax ymin ymax zmin zmax 
    All object is lung nodule,so we just not label that
    '''

    nodule_path=Path(DirectoryPath/"nodule")
    context_nodule_path=Path(DirectoryPath/"context_nodule")
    position_path=Path(DirectoryPath/'point_cloud')
    detection_path=Path(DirectoryPath/'detection')



    for index in range(len(slice_path_list)):
        img_nii = sitk.ReadImage(slice_path_list[index])
        img_array = sitk.GetArrayFromImage(img_nii)

        mask_nii = sitk.ReadImage(mask_path_list[index])
        mask_array = sitk.GetArrayFromImage(mask_nii)

        res=np.where(mask_array==1)#返回的是 x y z 轴,x y z 分别对应不同的轴方向，未必是原来那样的
        valid_z_pathes=list(dict(Counter(res[0])).keys())
        nodule_start=classify_nodule_and_relabel(valid_z_pathes,img_array,mask_array)

        for i in range(1,nodule_start+1):
            zlist, ylist, xlist = np.where(mask_array==i)
            position=np.argwhere(mask_array==1)    
            position=position/np.array(mask_array.shape)[:,None].T#用来对点云坐标数据进行归一化
            #就不存储类别了，因为所有的点云的类别都是肺结节
            position_path=nodule_path/f"position{index}_{i}.npy"
            save_np_array(position,position_path)#保存点云坐标

            xmin,xmax,ymin,ymax,zmin,zmax= xlist[0],xlist[-1],ylist[0],ylist[-1],zlist[0],zlist[-1]
            detection_label=np.array([xmin,xmax,ymin,ymax,zmin,zmax])
            detection_path=detection_path/f"detection{index}_{i}.npy"
            save_np_array(detection_label,detection_path)#保存检测坐标
            
            cropped_nodule = img_array[zmin:zmax+1, ymin:ymax+1, xmin:xmax+1]
            (context_xmin,context_xmax),(context_ymin,context_ymax),(context_zmin,context_zmax)=get_context_nodule_coordinate(xmin,xmax,ymin,ymax,zmin,zmax,ratio)
            cropped_nodule1 = img_array[context_zmin:context_zmax,context_ymin:context_ymax+1,context_xmin:context_xmax+1]

            # todo 3D展示结节
            # todo 存储结节（存储为.nii)
            #print('first saved')
            cropped_nodule_path =nodule_path/ f"nodule{index}_{i}.nii" 
            nodule_img = sitk.GetImageFromArray(cropped_nodule)
            sitk.WriteImage(nodule_img, cropped_nodule_path)

            cropped_nodule_path = context_nodule_path/f"nodule{index}_{i}.nii" 
            nodule_img = sitk.GetImageFromArray(cropped_nodule1)
            sitk.WriteImage(nodule_img, cropped_nodule_path)







In [ ]:
if __name__ == '__main__':
    directory = r"C:\Users\wyh196646\Desktop\test\resistant mutation\*"
    case_list = glob.glob(directory)
    run_multi_process(case_list, 50, partial(data_processing,dcm_to_nii=True), with_proc_num=True)
    print('all data have been processed')
